# Exploratory Data Analysis (EDA)

This notebook conducts exploratory data analysis on the harmonized and model-ready emergency incident datasets for Toronto and New York City. The objective of this analysis is to examine the distribution and variability of emergency response times, identify temporal and operational patterns associated with peak demand and delayed responses, and assess service-level performance beyond simple averages. Particular attention is given to tail delays and response-time threshold breaches, which are critical for understanding operational risk in emergency response systems. The findings from this EDA are used to guide feature engineering, model selection, and comparative analysis in subsequent stages of the project.



## 0. Import Libraries

In [0]:
# PySpark core
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col,
    sum as spark_sum,
    count,
    when,
    hour,
    dayofweek,
    date_format
)
from pyspark.sql import functions as F
# Optional: for local conversion & plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.style.use("default")
sns.set_context("notebook")

## 1. Sanity & Structure Check
Goal: Make sure the tables are truly “model-ready”.

- Row count (compare Toronto vs NYC scale)
- Column list & data types
- Missing values per column
- Duplicate incidents (by incident ID + timestamp)

### 1.1 Load Tables

In [0]:
toronto_df = spark.table("workspace.capstone_project.toronto_model_ready")
nyc_df = spark.table("workspace.capstone_project.nyc_model_ready")

### 1.2 Row Count (Scale Comparison)

In [0]:
toronto_count = toronto_df.count()
nyc_count = nyc_df.count()

toronto_count, nyc_count

**Row Count Summary**

The Toronto dataset contains 349,198 emergency incidents, while the New York City dataset contains 1,060,771 incidents. The substantially larger volume of incidents in New York City is expected due to differences in population size, urban density, and emergency service demand. These scale differences are taken into account during exploratory analysis and modeling, particularly when comparing response-time distributions and service-level risk across cities.

### 1.3 Column List & Data Types

In [0]:
toronto_df.printSchema()

In [0]:
nyc_df.printSchema()

In [0]:
set(toronto_df.columns) - set(nyc_df.columns), set(nyc_df.columns) - set(toronto_df.columns)

**Schema Consistency Check**

A comparison of column names across the Toronto and New York City datasets shows no differences in schema. Both datasets contain identical sets of analytical features, confirming that the data harmonization process successfully aligned the structure of the two datasets and enables direct cross-city comparison.

### 1.4 Missing Value per Column

In [0]:
def missing_value_summary(df):
    return df.select([
        spark_sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ])

In [0]:
def missing_table(df):
    total = df.count()
    m = missing_value_summary(df).toPandas().T.reset_index()
    m.columns = ["column_name", "missing_count"]
    m["missing_pct"] = m["missing_count"] / total * 100
    return m.sort_values("missing_count", ascending=False)

display(missing_table(toronto_df))
display(missing_table(nyc_df))

### 1.5 Duplicate Values Check

In [0]:
toronto_dupes = (
    toronto_df
    .groupBy("incident_id")
    .count()
    .filter(F.col("count") > 1)
)

print("Toronto duplicate incident_id count:", toronto_dupes.count())
display(toronto_dupes.orderBy(F.desc("count")).limit(20))

In [0]:
nyc_dupes = (
    nyc_df
    .groupBy("incident_id")
    .count()
    .filter(F.col("count") > 1)
)

print("NYC duplicate incident_id count:", nyc_dupes.count())
display(nyc_dupes.orderBy(F.desc("count")).limit(20))


### 1.6 Summary of Data Sanity & Structure

A series of sanity and structural checks were performed on the model-ready datasets for Toronto and New York City to ensure suitability for exploratory analysis and downstream modeling.

**Schema and Duplicates**  
Both datasets share an identical schema with consistent data types across all analytical fields. No duplicate records were detected in either dataset when grouped by `incident_id`, confirming one-to-one representation of emergency incidents.

**Missing Values**  
All feature variables are fully populated in both datasets. Missing values are observed only in the target variable `response_minutes`:

- **Toronto:** 12,469 missing values (3.45%)
- **New York City:** 422,625 missing values (28.49%)

The higher proportion of missing response times in the NYC dataset reflects a substantial number of incidents without an observed response completion time, while Toronto exhibits a much smaller fraction of such cases. These missing values are retained intentionally and are interpreted as censored observations, enabling subsequent survival analysis.

**Data Readiness**  
No unintended row filtering, duplication, or imputation was identified during data preparation. The datasets are therefore confirmed to be structurally sound and analytically ready for:
- distributional and tail-risk analysis using completed incidents, and  
- censor-aware survival modeling using the `event_indicator` field.

Overall, the model-ready datasets provide a reliable and consistent foundation for comparative analysis of emergency response performance across Toronto and New York City.

## 2. Target Variable Exploration
(Assuming response time or delay-based target)

- Distribution (histogram / KDE)
- Summary stats (mean, median, P90, P95)
- Skewness & outliers
- % of incidents breaching SLA thresholds (e.g. > X minutes)

Distributions and summary statistics below use completed incidents only
i.e. response_minutes IS NOT NULL.
<br>Censored cases are handled separately in survival analysis

### 2.1 Define Completed Incidents Subsets

In [0]:
toronto_complete = toronto_df.filter(F.col("response_minutes").isNotNull())
nyc_complete     = nyc_df.filter(F.col("response_minutes").isNotNull())

In [0]:
print("Toronto completed:", toronto_complete.count(), "/", toronto_df.count())
print("NYC completed:", nyc_complete.count(), "/", nyc_df.count())

### 2.2 Distribution: Histogram (KDE)

In [0]:
toronto_pd = toronto_complete.select("response_minutes").sample(fraction=0.2, seed=42).toPandas()
nyc_pd = nyc_complete.select("response_minutes").sample(fraction=0.2, seed=42).toPandas()
fig, axes = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

sns.histplot(
    toronto_pd["response_minutes"],
    bins=50, kde=True, ax=axes[0]
)
axes[0].set_title("Toronto Response Time Distribution (Completed Incidents)")
axes[0].set_ylabel("Count")

sns.histplot(
    nyc_pd["response_minutes"],
    bins=50, kde=True, ax=axes[1]
)
axes[1].set_title("NYC Response Time Distribution (Completed Incidents)")
axes[1].set_xlabel("Response Minutes")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.show()

**Response Time Distributions (Completed Incidents)**

The response time distributions for both Toronto and New York City are strongly right-skewed, indicating that while most incidents are handled within a relatively short time window, a non-trivial fraction experience substantially longer delays.

Toronto’s distribution exhibits a pronounced peak around the central response range, followed by a long and heavy right tail. This aligns with the high skewness value observed earlier and reflects the presence of extreme delayed responses that are not captured by average response times.

NYC shows a similarly right-skewed pattern but with a broader spread and a longer tail extending to higher response times. Compared to Toronto, NYC displays greater dispersion and a higher frequency of longer delays, consistent with its higher SLA breach rates and outlier prevalence.

Overall, these distributions reinforce that response time behavior in both cities is dominated by tail risk, motivating the use of percentile-based metrics, outlier analysis, and survival-based modeling rather than reliance on mean response times alone.

### 2.3 Summary Statistics (Mean, Median, P90, P95)

Helper Function

In [0]:
def response_summary(df):
    return df.select(
        F.round(F.mean("response_minutes"),4).alias("mean"),
        F.round(F.expr("percentile_approx(response_minutes, 0.5)"),4).alias("median"),
        F.round(F.expr("percentile_approx(response_minutes, 0.9)"),4).alias("p90"),
        F.round(F.expr("percentile_approx(response_minutes, 0.95)"),4).alias("p95"),
    )

In [0]:
# Compute summaries
toronto_stats = response_summary(toronto_complete).first()
nyc_stats     = response_summary(nyc_complete).first()

# Create Spark DataFrame
summary_df = spark.createDataFrame(
    [
        ("Toronto", toronto_stats["mean"], toronto_stats["median"],
         toronto_stats["p90"], toronto_stats["p95"]),
        ("NYC", nyc_stats["mean"], nyc_stats["median"],
         nyc_stats["p90"], nyc_stats["p95"]),
    ],
    ["city", "mean", "median", "p90", "p95"]
)

# Round for readability
summary_df = (
    summary_df
    .withColumn("mean", F.round("mean", 2))
    .withColumn("median", F.round("median", 2))
    .withColumn("p90", F.round("p90", 2))
    .withColumn("p95", F.round("p95", 2))
)

display(summary_df)

**Response Time Summary Statistics (Completed Incidents)**

Summary statistics further highlight the right-skewed nature of response-time distributions in both cities. In Toronto, the mean response time (5.33 minutes) exceeds the median (5.10 minutes), with high-percentile values reaching 7.67 minutes at P90 and 8.80 minutes at P95. New York City exhibits consistently higher values across all metrics, with a mean of 5.88 minutes, a median of 5.50 minutes, and substantially higher tail percentiles (P90 = 8.68 minutes, P95 = 10.22 minutes).

The divergence between median and high-percentile response times indicates that a relatively small fraction of delayed incidents disproportionately influences overall performance. The higher P90 and P95 values observed in NYC align with its greater skewness, higher outlier prevalence, and elevated SLA breach rates, underscoring more pronounced tail risk compared to Toronto.

### 2.4 Skewness & Outliers

#### 2.4.1 Skewness

In [0]:
# Compute skewness values
toronto_skew = toronto_complete.select(F.skewness("response_minutes")).first()[0]
nyc_skew     = nyc_complete.select(F.skewness("response_minutes")).first()[0]

# Create Spark DataFrame
skewness_df = spark.createDataFrame(
    [
        ("Toronto", toronto_skew),
        ("NYC", nyc_skew),
    ],
    ["city", "response_minutes_skewness"]
)

# Round for readability
skewness_df = skewness_df.withColumn(
    "response_minutes_skewness",
    F.round("response_minutes_skewness", 4)
)

display(skewness_df)

**Skewness of Response Time Distributions**

Both cities exhibit positively skewed response-time distributions, confirming the presence of long right tails. Toronto shows markedly higher skewness (4.78), indicating a heavier concentration of extreme delayed responses relative to its central tendency. New York City displays more moderate skewness (1.27), suggesting less extreme but still asymmetric response-time behavior.

Despite NYC exhibiting higher mean and high-percentile response times, Toronto’s stronger skewness indicates that its distribution is more sharply peaked with rarer but more extreme delay events. Together with the outlier and SLA breach analyses, these results demonstrate that response-time performance in both cities is driven by tail behavior rather than average outcomes.

#### 2.4.2 Outlier Inspection (IQR-based, diagnostic only)

In [0]:
# Function to compute outlier bounds


def outlier_profile(df):
    # bounds from your existing function
    q1, q3, lower, upper = outlier_bounds(df)

    stats = (
        df.select(
            F.count("*").alias("n_total"),
            F.sum((F.col("response_minutes") < lower).cast("int")).alias("n_lower_outliers"),
            F.sum((F.col("response_minutes") > upper).cast("int")).alias("n_upper_outliers"),
        )
        .withColumn("n_outliers", F.col("n_lower_outliers") + F.col("n_upper_outliers"))
        .withColumn("pct_outliers", F.col("n_outliers") / F.col("n_total") * 100)
    ).first()

    return {
        "Q1": q1, "Q3": q3, "lower_bound": lower, "upper_bound": upper,
        "n_total": stats["n_total"],
        "n_lower_outliers": stats["n_lower_outliers"],
        "n_upper_outliers": stats["n_upper_outliers"],
        "n_outliers": stats["n_outliers"],
        "pct_outliers": stats["pct_outliers"],
    }

tor = outlier_profile(toronto_complete)
nyc = outlier_profile(nyc_complete)

outlier_full_df = spark.createDataFrame(
    [
        ("Toronto", tor["n_outliers"], tor["pct_outliers"], tor["Q1"], tor["Q3"], tor["lower_bound"], tor["upper_bound"], tor["n_total"], tor["n_lower_outliers"], tor["n_upper_outliers"] ),
        ("NYC", nyc["n_outliers"], nyc["pct_outliers"], nyc["Q1"], nyc["Q3"], nyc["lower_bound"], nyc["upper_bound"],
         nyc["n_total"], nyc["n_lower_outliers"], nyc["n_upper_outliers"]),
    ],
    ["city",  "n_outliers", "pct_outliers", "Q1", "Q3", "lower_bound", "upper_bound",
     "n_total", "n_lower_outliers", "n_upper_outliers"]
)

outlier_full_df = (
    outlier_full_df
    .withColumn("Q1", F.round("Q1", 2))
    .withColumn("Q3", F.round("Q3", 2))
    .withColumn("lower_bound", F.round("lower_bound", 2))
    .withColumn("upper_bound", F.round("upper_bound", 2))
    .withColumn("pct_outliers", F.round("pct_outliers", 2))
)

display(outlier_full_df)



**Outlier Analysis (IQR Method, Completed Incidents)**

Outlier thresholds were defined using the interquartile range (IQR) method. Both Toronto and NYC exhibit a small but non-negligible proportion of response times exceeding the upper outlier bound, reflecting heavy right-tailed delay behavior.

- **Toronto:**  
  The upper outlier threshold is 9.58 minutes. Approximately 11,249 incidents exceed this threshold, representing 4.07% of completed incidents. Lower-bound outliers are rare and likely reflect minor timestamp irregularities rather than meaningful early responses.

- **New York City:**  
  The upper outlier threshold is higher at 10.68 minutes, with 43,467 incidents classified as upper outliers (4.20%). Similar to Toronto, lower-bound outliers are minimal.

Despite differences in absolute thresholds and incident volume, both cities show a comparable proportion of extreme delays. These results reinforce the presence of substantial tail risk in emergency response times and motivate the use of tail-sensitive metrics and censor-aware modeling rather than reliance on average response times alone.

**Note: Do not remove outliers — long delays are operationally meaningful**.

### 2.5 Service Level Agreement(SLA) Breach Analysis
SLA breach analysis measures the share of incidents for which response times exceed selected time thresholds. These thresholds represent practical performance benchmarks rather than strict policy guarantees. 

By examining breach rates, the analysis highlights delayed responses that are masked by average response times and provides a clearer view of operational risk during high-demand or constrained conditions.



In [0]:
SLA_1 = 5    # minutes
SLA_2 = 8    # minutes

Helper Function

In [0]:
def sla_breach_pct(df, threshold):
    return (
        df.select(
            (F.sum((F.col("response_minutes") > threshold).cast("int")) / F.count("*") * 100)
            .alias("pct")
        )
        .first()["pct"]
    )

Compute

In [0]:
tor_5 = sla_breach_pct(toronto_complete, SLA_1)
tor_8 = sla_breach_pct(toronto_complete, SLA_2)

nyc_5 = sla_breach_pct(nyc_complete, SLA_1)
nyc_8 = sla_breach_pct(nyc_complete, SLA_2)

sla_df = spark.createDataFrame(
    [
        ("Toronto", SLA_1, tor_5),
        ("Toronto", SLA_2, tor_8),
        ("NYC", SLA_1, nyc_5),
        ("NYC", SLA_2, nyc_8),
    ],
    ["city", "sla_threshold_minutes", "pct_breach"]
)

sla_df = sla_df.withColumn(
    "pct_breach", F.round("pct_breach", 2)
)
sla_pivot_df = (
    sla_df
    .groupBy("city")
    .pivot("sla_threshold_minutes")
    .agg(F.first("pct_breach"))
    .orderBy("city")
)

sla_pivot_df = sla_pivot_df.selectExpr(
    "city",
    "`5` as pct_over_5min",
    "`8` as pct_over_8min"
)

display(sla_pivot_df)

**SLA Breach Results**

SLA breach analysis reveals substantial differences in response-time reliability between Toronto and New York City. Using benchmark thresholds of 5 and 8 minutes, both cities exhibit high breach rates at stricter thresholds, indicating that delayed responses are common rather than exceptional.

- At the **5-minute threshold**, approximately **61.05%** of NYC incidents exceed the benchmark, compared to **52.25%** in Toronto.
- At the **8-minute threshold**, breach rates drop substantially but remain non-trivial, with **13.99%** of NYC incidents and **8.03%** of Toronto incidents exceeding this level.

Across both thresholds, NYC consistently exhibits higher breach rates, suggesting greater tail risk in response times. These findings reinforce earlier evidence from skewness and outlier analyses that average response times mask meaningful operational delays, and that tail-sensitive metrics are essential for evaluating emergency response performance.

### 2.6 Censoring Awareness (For Survival Analysis)
Censoring Validation

In [0]:
# Toronto
toronto_censoring = (
    toronto_df
    .agg(
        F.count("*").alias("n_total"),
        F.sum((F.col("event_indicator") == 1).cast("int")).alias("n_completed"),
        F.sum((F.col("event_indicator") == 0).cast("int")).alias("n_censored")
    )
    .withColumn("pct_censored", F.round(F.col("n_censored") / F.col("n_total") * 100, 2))
    .withColumn("city", F.lit("Toronto"))
)

# NYC
nyc_censoring = (
    nyc_df
    .agg(
        F.count("*").alias("n_total"),
        F.sum((F.col("event_indicator") == 1).cast("int")).alias("n_completed"),
        F.sum((F.col("event_indicator") == 0).cast("int")).alias("n_censored")
    )
    .withColumn("pct_censored", F.round(F.col("n_censored") / F.col("n_total") * 100, 2))
    .withColumn("city", F.lit("NYC"))
)

censoring_summary = toronto_censoring.unionByName(nyc_censoring)

display(censoring_summary.select(
    "city", "n_total", "n_completed", "n_censored", "pct_censored"
))



**Censoring Summary**

The censoring structure differs substantially between Toronto and New York City. In Toronto, 12,469 incidents (3.45%) are censored, indicating that the vast majority of incidents have an observed response completion time. In contrast, NYC exhibits a much higher degree of censoring, with 422,625 incidents (28.49%) lacking an observed response time.

This divergence reflects structural and operational differences in data recording and incident resolution across the two cities. For descriptive and distributional analyses, only completed incidents were used to ensure accurate characterization of observed response-time behavior. However, censored incidents are intentionally retained in the model-ready datasets and explicitly modeled using the `event_indicator` variable in subsequent survival analysis.

Accounting for censoring is therefore essential for valid cross-city comparison and for avoiding bias that would arise from analyzing completed incidents alone, particularly in the NYC dataset.

### 2.7 Summary of Target Variable Exploration

Exploratory analysis of the response time target reveals strongly right-skewed distributions in both Toronto and New York City, with long tails driven by a minority of substantially delayed incidents. Mean response times exceed median values, and high-percentile metrics (P90 and P95) indicate pronounced tail risk that is not captured by average performance measures alone. Outlier and SLA breach analyses further confirm that delayed responses are operationally meaningful and occur with non-trivial frequency in both cities, particularly in NYC.

Censoring is an important feature of the data, with a small proportion of censored incidents in Toronto (3.45%) and a substantially larger share in NYC (28.49%). To ensure valid interpretation, distributional analyses were conducted using completed incidents only, while censored cases are retained for subsequent survival-based modeling. Together, these findings motivate the use of tail-sensitive and censor-aware analytical approaches in the modeling stages that follow.

## 3. Temporal Patterns
Create / validate:
- Hour of day
- Day of week
- Month / season
- Weekend vs weekday

Explore:
- Avg & P90 response time by hour
- Incident volume by hour
- Heatmap: hour × day_of_week

**Important scope note (keep this logic consistent):**

- Response-time statistics → completed incidents only
- Incident volume → all incidents (completed + censored)

**P90** is the response time within which 90% of incidents are completed, highlighting delays in the slowest 10% of cases.

P90 refers to the response time value below which 90% of incidents fall, rather than the average response time of the fastest 90% of incidents. As a percentile-based metric, P90 captures the boundary of slower response behavior, whereas trimmed means summarize typical performance after excluding extreme delays.


### 3.1 Validate Temporal Features
- hour (0–23)
- day_of_week (1=Sunday … 7=Saturday)
- month (1–12)
- season

In [0]:
toronto_temporal = (
    toronto_df.select(
        F.lit("Toronto").alias("city"),
        F.min("hour").alias("min_hour"),
        F.max("hour").alias("max_hour"),
        F.min("day_of_week").alias("min_dow"),
        F.max("day_of_week").alias("max_dow"),
        F.min("month").alias("min_month"),
        F.max("month").alias("max_month")
    )
)

nyc_temporal = (
    nyc_df.select(
        F.lit("NYC").alias("city"),
        F.min("hour").alias("min_hour"),
        F.max("hour").alias("max_hour"),
        F.min("day_of_week").alias("min_dow"),
        F.max("day_of_week").alias("max_dow"),
        F.min("month").alias("min_month"),
        F.max("month").alias("max_month")
    )
)

temporal_validation_df = (
    toronto_temporal
    .unionByName(nyc_temporal)
    .toPandas()
    .set_index("city")   # make city the column header anchor
    .T                   # transpose
    .reset_index()
)

temporal_validation_df.rename(columns={"index": "metric"}, inplace=True)

display(temporal_validation_df)


Temporal validation confirms that hour, day-of-week, and month variables fall within expected ranges for both Toronto and NYC, indicating correct temporal encoding and readiness for downstream temporal pattern analysis.

### 3.2 Average & P90 Response Time by Hour
Use completed incidents only. 

Helper Function

In [0]:
def hourly_response_stats(df):
    return (
        df.filter(F.col("response_minutes").isNotNull())
          .groupBy("hour")
          .agg(
              F.round(F.mean("response_minutes"), 2).alias("avg_response"),
              F.round(F.expr("percentile_approx(response_minutes, 0.9)"), 2).alias("p90_response")
          )
          .orderBy("hour")
    )

#### 3.2.1 Average Respone by hour (Toronto vs NYC)

In [0]:
tor = hourly_response_stats(toronto_df).withColumn("city", F.lit("Toronto"))
nyc = hourly_response_stats(nyc_df).withColumn("city", F.lit("NYC"))

hourly_combined = tor.unionByName(nyc)

avg_pivot = (
    hourly_combined
    .groupBy("hour")
    .pivot("city", ["Toronto", "NYC"])
    .agg(F.first("avg_response"))
    .orderBy("hour")
)

display(avg_pivot)



#### 3.2.1 P90 response by hour (Toronto vs NYC)
**P90** is the response time within which 90% of incidents are completed, highlighting delays in the slowest 10% of cases.


In [0]:
p90_pivot = (
    hourly_combined
    .groupBy("hour")
    .pivot("city", ["Toronto", "NYC"])
    .agg(F.first("p90_response"))
    .orderBy("hour")
)

display(p90_pivot)

#### Hourly Response-Time Patterns Summary

Hourly analysis reveals clear diurnal patterns in both cities. Average response times are lowest during daytime and evening hours and increase during late-night and early-morning periods, with NYC consistently exhibiting higher average response times than Toronto across all hours. While average differences are modest (approximately 0.3–0.6 minutes), tail behavior differs more substantially.

P90 response times show pronounced overnight and early-morning delays, particularly in NYC, where the slowest 10% of responses exceed Toronto’s P90 by more than one minute during several off-peak hours. These patterns indicate that response-time risk is driven less by typical daytime operations and more by reduced overnight capacity and elevated tail delays, especially in NYC.


### 3.3 Incident Volumne by Hour
Volume includes all incidents, regardless of completion.

In [0]:
def hourly_volume(df):
    return (
        df.groupBy("hour")
          .count()
          .withColumnRenamed("count", "incident_volume")
          .orderBy("hour")
    )


In [0]:
toronto_hourly_vol = (
    toronto_df
    .groupBy("hour")
    .count()
    .withColumn("city", F.lit("Toronto"))
)

nyc_hourly_vol = (
    nyc_df
    .groupBy("hour")
    .count()
    .withColumn("city", F.lit("NYC"))
)

hourly_volume_combined = toronto_hourly_vol.unionByName(nyc_hourly_vol)

hourly_volume_pivot = (
    hourly_volume_combined
    .groupBy("hour")
    .pivot("city", ["Toronto", "NYC"])
    .agg(F.first("count"))
    .orderBy("hour")
)

display(hourly_volume_pivot)

**Incident Volume by Hour Summary**

Incident volume exhibits a strong diurnal pattern in both Toronto and New York City. Call volumes are lowest during late-night and early-morning hours (approximately 02:00–05:00) and increase steadily throughout the day, peaking during late afternoon and early evening. NYC consistently experiences substantially higher incident volumes than Toronto at every hour, often by a factor of three to four.

The temporal alignment between peak incident volume and elevated response-time levels suggests that demand intensity is an important driver of response-time variation. However, the presence of higher response-time tail risk during overnight hours—despite lower call volumes—indicates that capacity constraints and staffing availability likely play a more significant role during off-peak periods.


### 3.4 Heatmap for  Hours x Day of Week

Prepare heatmap data (only for completed incidents)

In [0]:
def heatmap_data(df):
    return (
        df.filter(F.col("response_minutes").isNotNull())
          .groupBy("day_of_week", "hour")
          .agg(F.round(F.mean("response_minutes"), 2).alias("avg_response"))
          .orderBy("day_of_week", "hour")
    )

In [0]:
# Convert to Pandas for plotting:
toronto_heat_pd = heatmap_data(toronto_df).toPandas()
nyc_heat_pd     = heatmap_data(nyc_df).toPandas()

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# --- Toronto (top) ---
sns.heatmap(
    toronto_heat_pd.pivot(
        index="day_of_week",
        columns="hour",
        values="avg_response"
    ),
    ax=axes[0],
    cmap="YlOrRd"
)
axes[0].set_title("Toronto: Avg Response Time by Hour × Day of Week")
axes[0].set_ylabel("Day of Week")

# --- NYC (bottom) ---
sns.heatmap(
    nyc_heat_pd.pivot(
        index="day_of_week",
        columns="hour",
        values="avg_response"
    ),
    ax=axes[1],
    cmap="YlOrRd"
)
axes[1].set_title("NYC: Avg Response Time by Hour × Day of Week")
axes[1].set_xlabel("Hour of Day")
axes[1].set_ylabel("Day of Week")

plt.tight_layout()
plt.show()


**Hour × Day-of-Week Response-Time Patterns**

The heatmaps reveal clear and consistent temporal structure in both cities. Average response times are highest during overnight and early-morning hours (approximately 00:00–06:00) across most days of the week, with gradual improvement during daytime hours. Toronto exhibits relatively stable response times throughout the week, with modest weekday–weekend variation.

In contrast, NYC shows uniformly higher response times across all hours, with more pronounced overnight delays and less recovery during daytime periods. The persistence of elevated response times during low-demand overnight hours suggests that capacity constraints, staffing levels, or operational coverage—rather than demand alone—play a key role in shaping response-time performance, particularly in NYC.


### 3.5 Weekend vs Weekday Analysis

In [0]:
def add_weekend_flag(df):
    return df.withColumn(
        "is_weekend",
        F.when(F.col("day_of_week").isin(1, 7), "Weekend").otherwise("Weekday")
    )

toronto_wd = add_weekend_flag(toronto_df)
nyc_wd     = add_weekend_flag(nyc_df)

In [0]:
# Toronto
toronto_weekend_stats = (
    toronto_wd
    .filter(F.col("response_minutes").isNotNull())
    .groupBy("is_weekend")
    .agg(
        F.round(F.mean("response_minutes"), 2).alias("avg_response"),
        F.round(F.expr("percentile_approx(response_minutes, 0.9)"), 2).alias("p90_response")
    )
    .withColumn("city", F.lit("Toronto"))
)

# NYC
nyc_weekend_stats = (
    nyc_wd
    .filter(F.col("response_minutes").isNotNull())
    .groupBy("is_weekend")
    .agg(
        F.round(F.mean("response_minutes"), 2).alias("avg_response"),
        F.round(F.expr("percentile_approx(response_minutes, 0.9)"), 2).alias("p90_response")
    )
    .withColumn("city", F.lit("NYC"))
)

# Combine
weekend_comparison = (
    toronto_weekend_stats
    .unionByName(nyc_weekend_stats)
    .select("city", "is_weekend", "avg_response", "p90_response")
    .orderBy("city", "is_weekend")
)

display(weekend_comparison)


In [0]:
plt.figure(figsize=(8, 5))

ax = sns.barplot(
    data=weekend_pd,
    x="is_weekend",
    y="p90_response",
    hue="city"
)

ax.set_title("P90 Response Time: Weekday vs Weekend", fontsize=12)
ax.set_xlabel("Day Type")
ax.set_ylabel("P90 Response Time (minutes)")

# Add value labels on bars
for container in ax.containers:
    ax.bar_label(container, fmt="%.2f", padding=0.5)

# Move legend outside
ax.legend(
    title="City",
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0
)

plt.tight_layout()
plt.show()


In [0]:
plt.figure(figsize=(8, 5))

ax = sns.barplot(
    data=weekend_pd,
    x="is_weekend",
    y="avg_response",
    hue="city"
)

ax.set_title("Average Response Time: Weekday vs Weekend", fontsize=12)
ax.set_xlabel("Day Type")
ax.set_ylabel("Average Response Time (minutes)")

# Add value labels on bars
for container in ax.containers:
    ax.bar_label(container, fmt="%.2f", padding=0.5)

# Move legend outside
ax.legend(
    title="City",
    bbox_to_anchor=(1.02, 1),
    loc="upper left",
    borderaxespad=0
)

plt.tight_layout()
plt.show()

**Weekday vs Weekend Response-Time Comparison**

Both Toronto and New York City exhibit slightly lower average response times on weekends compared to weekdays, consistent with reduced traffic congestion and lower overall incident demand during non-working days. This improvement is reflected not only in average response times but also in high-percentile (P90) values, indicating that weekend conditions influence response performance broadly across the distribution.

The gap between average and P90 response times in both cities reflects a right-skewed response-time distribution, where a small proportion of incidents experience substantially longer delays. However, the similarity in weekday–weekend patterns across both metrics suggests that weekend effects do not disproportionately alter extreme response delays, but instead provide modest, uniform improvements across typical and slower response cases.



### 3.6 Summary of Temporal Patterns
Temporal analysis reveals strong diurnal and weekly structure in emergency response performance across both cities. Response times and incident volumes vary systematically by hour of day, with slower responses occurring during overnight and early-morning periods and higher volumes during daytime and evening hours. Weekend response times are slightly lower than weekday levels, consistent with reduced traffic and demand, and this pattern is observed uniformly across both average and high-percentile metrics.

Overall, temporal variation influences response performance across the entire distribution, highlighting the importance of accounting for time-of-day and day-of-week effects in subsequent modeling and risk analysis.


## 4. Spatial/ Operational Signals
**Toronto**
- Ward / Station Area
- Alarm level
- Call source

**NYC**
- Borough
- Incident type
- Alarm level

Explore:
- Response time by area (mean + tail)
- Volume vs delay by area
- High-volume ≠ fast response (important insight)

## 5. Incident Characteristic
- Incident type vs response time
- Alarm level vs response time
- Rare but high-risk categories

## 6. Cross-City Comparability Check (critical)
Before modeling:
- Are response-time definitions aligned?
- Same units? (seconds vs minutes)
- Similar feature engineering logic?

Create:
- Normalized response time distributions
- Percentile comparison (Toronto vs NYC)

## 7. Correlation & Leakage Scan
- Correlation matrix (numeric only)
- Watch for:
  - Features derived from response time
  - Post-arrival timestamps
- Flag anything suspicious early